<a href="https://colab.research.google.com/github/NguenDuyNhat/DuyNhat_CS2018_UIT/blob/master/POS_TAGGING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Truy cập vào nơi chứa data**

In [ ]:
cd /content/drive/My Drive/CoQUY/Data_coQuy/Kho ngu lieu 10000 cau duoc gan nhan tu loai

# **Khai báo các thư viện cần thiết**

In [ ]:
import os 
import pandas as pd
import joblib
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from gensim.models import KeyedVectors
import numpy as np

# **Load dữ liệu thô**

In [ ]:
raw_data=[]
for i in os.listdir("/content/drive/My Drive/CoQUY/Data_coQuy/Kho ngu lieu 10000 cau duoc gan nhan tu loai"):  
        with open(i) as f:
          raw_data.append(f.read())

In [ ]:
raw_data

# **Tách dữ liệu thô thành từng câu**

In [ ]:
data_X=[]
for i in range(len(raw_data)):
  b=raw_data[i].split('\n')
  data_X.append(b)

In [ ]:
X=[]
for i in data_X:
    for j in i:
        X.append(j)

In [ ]:
X

# **Có tất cả 10521 câu được gán sẵn nhãn**

In [ ]:
len(X)

# **Phân dữ liệu thành 3 tập: train,dev,test.**
# **Tập train chiếm 75% dữ liệu: 7365 câu**
# **Tập val chiếm 15% dữ liệu : 1578 câu** 
# **Tập test chiếm 10% dữ liệu: 1578 câu**

In [ ]:
X_test, X_train  = train_test_split(X, test_size=0.7,random_state=101)
X_test, X_val  = train_test_split(X_test, test_size=0.5,random_state=101) 

In [ ]:
print(len(X_train),len(X_test),len(X_val))

# **Xử lí các câu trong tập train thành dạng dataframe: 2 cột**
# **Cột thứ 1: là từ được tách ra từ câu**
# **Cột thứ 2: là nhãn của từ được gán**
# **Sau khi đưa tập train về dataframe, tiến hành lưu thành file train.csv: gồm 2 cột và 149946 dòng**

In [ ]:
data_train=[]
for senten in X_train:
    senten_split=senten.split()
    for wp in senten_split:
        data_train.append(wp)
words_train=[]
pos_tag_train=[]
for i in data_train:
    if (i.count("/")==1):
        index=i.split("/")
        if( index[-1].isalpha() or index[-1]=="(" or index[-1]==")" or index[-1]==":" or index[-1]=="." or index[-1]=="," or index[-1]== ";" or index[-1]=="?"  or index[-1]=="!" or index[-1]=="+" or index[-1]=="-" or index[-1]=="..." or index[-1]=="<" or index[-1]==">" or index[-1]=="{" or index[-1]=="}"):
            words_train.append(index[0]) # ( ) : . , ; ? ! + - ... < > { }
            pos_tag_train.append(index[-1])
df_train=pd.DataFrame(columns=("words","pos_tag"))
df_train["words"]=words_train
df_train["pos_tag"]=pos_tag_train
df_train.to_csv('/content/drive/MyDrive/CoQUY/train.csv')

# **Sau khi đưa tập validation về dataframe, tiến hành lưu thành file val.csv: gồm 2 cột và 33167 dòng**




In [ ]:
data_val=[]
for senten in X_val:
    senten_split=senten.split()
    for wp in senten_split:
        data_val.append(wp)
words_val=[]
pos_tag_val=[]
for i in data_val:
    if (i.count("/")==1):
        index=i.split("/")
        if( index[-1].isalpha() or index[-1]=="(" or index[-1]==")" or index[-1]==":" or index[-1]=="." or index[-1]=="," or index[-1]== ";" or index[-1]=="?"  or index[-1]=="!" or index[-1]=="+" or index[-1]=="-" or index[-1]=="..." or index[-1]=="<" or index[-1]==">" or index[-1]=="{" or index[-1]=="}"):
            words_val.append(index[0]) # ( ) : . , ; ? ! + - ... < > { }
            pos_tag_val.append(index[-1]) # ( ) : . , ; ? ! +
df_val=pd.DataFrame(columns=("words","pos_tag"))
df_val["words"]=words_val
df_val["pos_tag"]=pos_tag_val
df_val.to_csv('/content/drive/MyDrive/CoQUY/val.csv')

# **Sau khi đưa tập test về dataframe, tiến hành lưu thành file test.csv: gồm 2 cột và 32578 dòng**

In [ ]:
data_test=[]
for senten in X_test:
    senten_split=senten.split()
    for wp in senten_split:
        data_test.append(wp)
words_test=[]
pos_tag_test=[]
for i in data_test:
    if (i.count("/")==1):
        index=i.split("/")
        if( index[-1].isalpha() or index[-1]=="(" or index[-1]==")" or index[-1]==":" or index[-1]=="." or index[-1]=="," or index[-1]== ";" or index[-1]=="?"  or index[-1]=="!" or index[-1]=="+" or index[-1]=="-" or index[-1]=="..." or index[-1]=="<" or index[-1]==">" or index[-1]=="{" or index[-1]=="}"):
            words_test.append(index[0])
            pos_tag_test.append(index[-1])
df_test=pd.DataFrame(columns=("words","pos_tag"))
df_test["words"]=words_test
df_test["pos_tag"]=pos_tag_test
df_test.to_csv('/content/drive/MyDrive/CoQUY/test.csv')

# **Gọi Word Embedding**

In [ ]:
word_emb = KeyedVectors.load_word2vec_format("/content/drive/MyDrive/CoQUY/baomoi.model.bin", binary=True)

# **Đưa các từ ở df_train[words] về dạng thích hợp với mô hình để đưa vào huấn luyện=> X_features_train**
# **Đưa các nhãn => X_labels_train**

In [ ]:
X_features_train=[]
Y_labels_train=[]
def convert(x):
    return dictionary[x]

set_pos_train=set (pos_tag_train)
dictionary = {}

for idx, i in enumerate(set_pos_train):
    dictionary[i]=idx

#df_train["pos_tag"]=df_train["pos_tag"].apply(lambda x: convert(x))

for i in pos_tag_train:
    Y_labels_train.append(i)

for idx,w in enumerate(df_train["words"]):
    if (w in word_emb.vocab.keys()):
        X_features_train.append(word_emb.word_vec(w))
    else:
        X_features_train.append(np.zeros(400,dtype=float))

# **Đưa các từ ở df_val[words] về dạng thích hợp với mô hình để đưa vào huấn luyện => X_features_val**
# **Đưa các nhãn => X_labels_val**

In [ ]:
X_features_val=[]
Y_labels_val=[]
def convert(x):
    return dictionary[x]

set_pos_val=set (pos_tag_val)
dictionary = {}

for idx, i in enumerate(set_pos_val):
    dictionary[i]=idx

#df_val["pos_tag"]=df_val["pos_tag"].apply(lambda x: convert(x))

for i in pos_tag_val :
    Y_labels_val.append(i)

for idx,w in enumerate(df_val["words"]):
    if (w in word_emb.vocab.keys()):
        X_features_val.append(word_emb.word_vec(w))
    else:
        X_features_val.append(np.zeros(400,dtype=float))

# **Đưa các từ ở df_test[words] về dạng thích hợp với mô hình để đưa vào huấn luyện => X_features_test**
# **Đưa các nhãn => X_labels_test**

In [ ]:
X_features_test=[]
Y_labels_test=[]
def convert(x):
    return dictionary[x]

set_pos_test=set (pos_tag_test)
dictionary = {}

for idx, i in enumerate(set_pos_test):
    dictionary[i]=idx


for i in pos_tag_test :
    Y_labels_test.append(i)

for idx,w in enumerate(df_test["words"]):
    if (w in word_emb.vocab.keys()):
        X_features_test.append(word_emb.word_vec(w))
    else:
        X_features_test.append(np.zeros(400,dtype=float))

# **Xem lại kích thước ở mỗi features và label của mỗi tập**

In [ ]:
print(len(X_features_train),len(Y_labels_train))
print(len(X_features_val),len(Y_labels_val))
print(len(X_features_test),len(Y_labels_test))

# **Sử dụng mô hình SVM để train cho tập train gồm 149946 features và labels, sau đó dự đoán trên tập val**

In [ ]:
model1=SVC().fit(X_features_train,Y_labels_train)
predict=model1.predict(X_features_val)

# **Kết quả và confusion matrix khi mô hình train xong và dự đoán trên tập val**

In [ ]:
print('Accuracy: ',accuracy_score(Y_labels_val, predict))
print('Classification report: \n',classification_report(Y_labels_val, predict))
print('Confusion matrix: \n',confusion_matrix(Y_labels_val, predict))

# **Lưu mô hình để khi demo không cần phải train lại**

In [ ]:
filename = '/content/drive/MyDrive/CoQUY/model_SVM_result.sav' 
joblib.dump(model1, filename)

# **Load mô hình ra để sử dụng cho nhanh trong quá trình demo**

In [ ]:
model_loaded_result=joblib.load("/content/drive/MyDrive/CoQUY/model_SVM_result.sav")

# **Dùng mô hình đã train để dự đoán tập test và in ra kết quả**

In [ ]:
result_test=model_loaded_result.predict(X_features_test)
print('Accuracy: ',accuracy_score(Y_labels_test, result_test))
print('Classification report: \n',classification_report(Y_labels_test, result_test))
print('Confusion matrix: \n',confusion_matrix(Y_labels_test, result_test))

# **Demo: sử dụng công cụ tách từ từ pyvi, chạy đoạn code dưới để install pyvi sau đó chạy đoạn code cuối để nhập câu cần gán nhãn từ loại và sẽ được kết quả**

In [ ]:
! pip install pyvi
from pyvi import ViTokenizer

In [ ]:
sentence=input(str())
word_emb = KeyedVectors.load_word2vec_format("/content/drive/MyDrive/CoQUY/baomoi.model.bin", binary=True)
def raw_senten(sentence):
    list_senten=[]
    np_senten=[]
    list_senten=ViTokenizer.tokenize(sentence).split()
    return list_senten
def raw_word(list_senten):
    np_=[]
    for i in list_senten:
        if (i in word_emb.vocab.keys()):
            np_.append((word_emb.word_vec(i)).reshape(1,400))
        else:
            np_.append((np.zeros(400,dtype=float)).reshape(1,400))
    return np_
list_senten=raw_senten(sentence)
np_=raw_word(list_senten)
a=[]
for i in np_:
    a.append(model_loaded_result.predict(i))
#print(a)
for i in range (len(list_senten)):
    print((list_senten[i])+"/"+a[i][0],end=" ")